In [135]:
### NQF_WilsonDairy_HydrologicalMonitoring_v1.py
### Version: 6/2/2021
### Author: Khem So, khem_so@fws.gov, (503) 231-6839
### Abstract: This Python 3 script pulls data from the Wilson Dairy Hydrological Data V2 ArcGIS Online feature service (collected via Survey123) and summarizes the data to match an Excel template.

In [136]:
import arcpy
import pandas as pd
from arcgis import GIS
import time, os, fnmatch, shutil
import openpyxl

In [137]:
### ArcGIS Online stores date-time information in UTC by default. This function converts time zones and can be used to convert from UTC ("UTC") to localized Pacific time ("US/Pacific").
from datetime import datetime
from pytz import timezone
def change_timezone_of_field(df, source_date_time_field, new_date_time_field_suffix, source_timezone, new_timezone):
    """Returns the values in *source_date_time_field* with its timezone converted to a new timezone within a new field *new_date_time_field*
    : param df: The name of the spatially enabled or pandas DataFrame containing datetime fields
    : param source_date_time_field: The name of the datetime field whose timezone is to be changed
    : param new_date_time_field_suffix: Suffix appended to the end of the name of the source datetime field. This is used to create the new date time field name.
    : param source_timezone: The name of the source timezone
    : param new_timezone: The name of the converted timezone. For possible values, see https://gist.github.com/heyalexej/8bf688fd67d7199be4a1682b3eec7568
    """
    # Define the source timezone in the source_date_time_field
    df[source_date_time_field] = df[source_date_time_field].dt.tz_localize(source_timezone)
    # Define the name of the new date time field
    new_date_time_field = source_date_time_field + new_date_time_field_suffix
    # Convert the datetime in the source_date_time_field to the new timezone in a new field called new_date_time_field
    df[new_date_time_field] = df[source_date_time_field].dt.tz_convert(new_timezone)

In [138]:
def py_datetime_excel(df, source_date_time_field, new_date_time_field, format="%m/%d/%Y %H:%M:%S"):
    """Convert Python date time in *source_date_time_field* into format Excel can read more easily with the following format (%m/%d/%Y %H:%M:%S) by default. The new date time is contained within a new field *new_date_time_field*.
    : param df: The name of the spatially enabled or pandas DataFrame containing datetime fields
    : param source_date_time_field: The name of the datetime field whose timezone is to be changed
    : param new_date_time_field: The name of the new datetime field
    : format: The name of the date time format. For possible values, see https://strftime.org/
    """
    df[new_date_time_field] = df[source_date_time_field].dt.strftime(format)

In [139]:
### Allow authentication via login to U.S. Fish & Wildlife Service ArcGIS Online account via ArcGIS Pro
gis = GIS("pro")

In [140]:
### Enter path for local file saving
# uncomment next line to use ArcGIS interface, otherwise hard coding out_workspace
# out_workspace = arcpy.GetParameterAsText(0)
out_workspace = "C:/Users/kso/Desktop/"

In [141]:
### Create timestamp for file naming
t = time.localtime()
timestamp = time.strftime('%Y-%m-%d_%H%M', t)

In [142]:
### Paths to ArcGIS Online data
# To populate Service ItemId, go to Feature Service webpage and in bottom right corner, click on the View link.
# Current Feature Service webpage: https://fws.maps.arcgis.com/home/item.html?id=c6b4f8f33b804dea8c2232c3d7786e9f
ServiceItemID = gis.content.get("c6b4f8f33b804dea8c2232c3d7786e9f")

In [143]:
### There are separate methods for pulling spatial versus non-spatial data into Python. Spatial layers will become Spatially Enabled DataFrame objects. 
## Define variables pointing to spatial layers
WilsonHydroMonitoringLyr = ServiceItemID.layers[0]
## Create Spatially Enabled DataFrame objects
sedfHydroAllData = pd.DataFrame.spatial.from_layer(WilsonHydroMonitoringLyr)

In [144]:
list(sedfHydroAllData.columns)

['objectid',
 'globalid',
 'CreationDate',
 'Creator',
 'EditDate',
 'Editor',
 'observers',
 'observers_other',
 'todays_date',
 'Measurement_Type',
 'what_are_you_measuring_other',
 'creek_gauge_ft',
 'UTC_Time_Measured_4',
 'Location__logger',
 'UTC_Time_Measured_1',
 'depth_to_water_level_inside_wel',
 'depth_to_water_outside_well_ft',
 'depth_to_ground_level_inside_we',
 'depth_to_ground_outside_well_ft',
 'UTC_Time_Measured_2',
 'staff_gauge_ht_ft',
 'data_downloaded',
 'logger_serial',
 'time_logger_pulled_UTC_time',
 'time_logger_re_deployed_UTC_t',
 'battery',
 'Location_culvert',
 'Depth_to_H20_fr0m_top_culvrt_ft',
 'UTC_Time_Measured_3',
 'additional_notes',
 'Measurement_Type_other',
 'Location__logger_other',
 'SHAPE']

In [145]:
## Convert all datetime fields in dataframe from UTC to Pacific within new field with _Pacific suffix
for col in sedfHydroAllData.columns:
    if sedfHydroAllData[col].dtype == 'datetime64[ns]':
        change_timezone_of_field(sedfHydroAllData, col, "_Pacific", "UTC", "US/Pacific")

In [146]:
sedfHydroAllData.dtypes

objectid                                                      int64
globalid                                                     object
CreationDate                                    datetime64[ns, UTC]
Creator                                                      object
EditDate                                        datetime64[ns, UTC]
Editor                                                       object
observers                                                    object
observers_other                                              object
todays_date                                     datetime64[ns, UTC]
Measurement_Type                                             object
what_are_you_measuring_other                                 object
creek_gauge_ft                                              float64
UTC_Time_Measured_4                             datetime64[ns, UTC]
Location__logger                                             object
UTC_Time_Measured_1                             

In [166]:
for col in sedfHydroAllData.select_dtypes(include=['datetime64[ns, UTC]','datetime64[ns, US/Pacific]']):
    if sedfHydroAllData[col]
    py_datetime_excel(sedfHydroAllData, col, col)

In [165]:
sedfHydroAllData.select_dtypes(include=['datetime64[ns, UTC]','datetime64[ns, US/Pacific]'])


,CreationDate,EditDate,todays_date,UTC_Time_Measured_4,UTC_Time_Measured_1,UTC_Time_Measured_2,time_logger_pulled_UTC_time,time_logger_re_deployed_UTC_t,UTC_Time_Measured_3,CreationDate_Pacific,EditDate_Pacific,todays_date_Pacific,UTC_Time_Measured_4_Pacific,UTC_Time_Measured_1_Pacific,UTC_Time_Measured_2_Pacific,time_logger_pulled_UTC_time_Pacific,time_logger_re_deployed_UTC_t_Pacific,UTC_Time_Measured_3_Pacific
0,2020-02-19 01:27:35.359999895+00:00,2020-03-04 20:01:31.436000109+00:00,2020-02-18 20:00:00+00:00,2020-02-18 20:30:00+00:00,NaT,NaT,NaT,NaT,NaT,2020-02-18 17:27:35.359999895-08:00,2020-03-04 12:01:31.436000109-08:00,2020-02-18 12:00:00-08:00,2020-02-18 12:30:00-08:00,NaT,NaT,NaT,NaT,NaT
1,2020-02-19 01:27:41.535000086+00:00,2020-02-19 01:27:41.535000086+00:00,2020-02-18 20:00:00+00:00,NaT,NaT,2020-02-18 20:34:09.976000071+00:00,2020-02-18 20:33:00+00:00,2020-02-18 20:38:00+00:00,NaT,2020-02-18 17:27:41.535000086-08:00,2020-02-18 17:27:41.535000086-08:00,2020-02-18 12:00:00-08:00,NaT,NaT,2020-02-18 12:34:09.976000071-08:00,2020-02-18 12:33:00-08:00,2020-02-18 12:38:00-08:00,NaT
2,2020-02-19 01:27:44.867000103+00:00,2020-02-19 01:27:44.867000103+00:00,2020-02-18 20:00:00+00:00,NaT,NaT,NaT,2020-02-18 20:26:40.049999952+00:00,2020-02-18 20:27:00+00:00,NaT,2020-02-18 17:27:44.867000103-08:00,2020-02-18 17:27:44.867000103-08:00,2020-02-18 12:00:00-08:00,NaT,NaT,NaT,2020-02-18 12:26:40.049999952-08:00,2020-02-18 12:27:00-08:00,NaT
3,2020-02-19 01:27:48.403000116+00:00,2020-02-19 01:27:48.403000116+00:00,2020-02-18 20:00:00+00:00,NaT,NaT,NaT,2020-02-18 20:23:44.302999973+00:00,2020-02-18 20:24:00+00:00,NaT,2020-02-18 17:27:48.403000116-08:00,2020-02-18 17:27:48.403000116-08:00,2020-02-18 12:00:00-08:00,NaT,NaT,NaT,2020-02-18 12:23:44.302999973-08:00,2020-02-18 12:24:00-08:00,NaT
4,2020-02-19 01:27:55.072999954+00:00,2020-02-19 05:24:42.161000013+00:00,2020-02-18 20:00:00+00:00,NaT,2020-02-18 20:00:34.260999918+00:00,NaT,2020-02-18 20:03:00+00:00,2020-02-18 20:11:21.598999977+00:00,NaT,2020-02-18 17:27:55.072999954-08:00,2020-02-18 21:24:42.161000013-08:00,2020-02-18 12:00:00-08:00,NaT,2020-02-18 12:00:34.260999918-08:00,NaT,2020-02-18 12:03:00-08:00,2020-02-18 12:11:21.598999977-08:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,2021-04-17 14:19:16.963999987+00:00,2021-04-17 14:19:16.963999987+00:00,2021-04-15 19:00:00+00:00,2021-04-15 17:48:00+00:00,NaT,NaT,NaT,NaT,NaT,2021-04-17 07:19:16.963999987-07:00,2021-04-17 07:19:16.963999987-07:00,2021-04-15 12:00:00-07:00,2021-04-15 10:48:00-07:00,NaT,NaT,NaT,NaT,NaT
203,2021-04-26 04:31:53.681999922+00:00,2021-04-26 04:31:53.681999922+00:00,2021-04-26 02:14:19.845000029+00:00,2021-04-26 02:14:00+00:00,NaT,NaT,NaT,NaT,NaT,2021-04-25 21:31:53.681999922-07:00,2021-04-25 21:31:53.681999922-07:00,2021-04-25 19:14:19.845000029-07:00,2021-04-25 19:14:00-07:00,NaT,NaT,NaT,NaT,NaT
204,2021-04-26 23:38:29.552000046+00:00,2021-04-26 23:38:29.552000046+00:00,2021-04-26 02:21:06.861999989+00:00,NaT,NaT,NaT,NaT,NaT,2021-04-26 02:21:17.892999887+00:00,2021-04-26 16:38:29.552000046-07:00,2021-04-26 16:38:29.552000046-07:00,2021-04-25 19:21:06.861999989-07:00,NaT,NaT,NaT,NaT,NaT,2021-04-25 19:21:17.892999887-07:00
205,2021-04-26 23:38:36.173000097+00:00,2021-04-26 23:38:36.173000097+00:00,2021-04-26 02:23:46.069000006+00:00,NaT,NaT,NaT,NaT,NaT,2021-04-26 02:23:00+00:00,2021-04-26 16:38:36.173000097-07:00,2021-04-26 16:38:36.173000097-07:00,2021-04-25 19:23:46.069000006-07:00,NaT,NaT,NaT,NaT,NaT,2021-04-25 19:23:00-07:00


In [132]:
## Convert all datetime fields into a format Excel can read more easily
py_datetime_excel(sedfHydroAllData, 'CreationDate_Pacific', 'CreationDate_Pacific')

In [114]:
## Subset data for Dempsey Creek Staff Gage at Wilson Dairy, Nisqually NWR
# Select data based on 2 conditions: measurement type = piezometer and location = dempsey creek gauge OR measurement type = dempsey creek level quick survey
sedfHydroStaffGageData = sedfHydroAllData.loc[((sedfHydroAllData['Measurement_Type'] == "piezometers") & (sedfHydroAllData['Location__logger'] == "dempsey_creek_gauge")) | ((sedfHydroAllData['Measurement_Type'] == "dempsey_creek_level_quick_surve"))]
# Select columns and reset in desired order
sedfHydroStaffGageData = sedfHydroStaffGageData[['todays_date_Pacific', 'UTC_Time_Measured_4_Pacific', 'observers', 'observers_other', 'creek_gauge_ft', 'UTC_Time_Measured_2_Pacific', 'staff_gauge_ht_ft', 'data_downloaded', 'logger_serial', 'time_logger_pulled_UTC_time_Pacific', 'time_logger_re_deployed_UTC_t_Pacific', 'battery', 'additional_notes', 'globalid', 'SHAPE']]
# Convert Python date time into format Excel can read more easily
sedfHydroStaffGageData['todays_date_Pacific'] = sedfHydroStaffGageData['todays_date_Pacific'].dt.strftime('%x')
sedfHydroStaffGageData['UTC_Time_Measured_4_Pacific'] = sedfHydroStaffGageData['UTC_Time_Measured_4_Pacific'].dt.strftime('%X')
sedfHydroStaffGageData['UTC_Time_Measured_2_Pacific'] = sedfHydroStaffGageData['UTC_Time_Measured_2_Pacific'].dt.strftime('%X')
sedfHydroStaffGageData['time_logger_pulled_UTC_time_Pacific'] = sedfHydroStaffGageData['time_logger_pulled_UTC_time_Pacific'].dt.strftime('%X')
sedfHydroStaffGageData['time_logger_re_deployed_UTC_t_Pacific'] = sedfHydroStaffGageData['time_logger_re_deployed_UTC_t_Pacific'].dt.strftime('%X')
# Rename columns
sedfHydroStaffGageData.rename(columns = {'todays_date_Pacific':'Date', 'UTC_Time_Measured_4_Pacific':'Time Gauge Reading (Pacific)', 'observers':'Observers', 'observers_other':'Other Observers', 'creek_gauge_ft':'Water Level (ft)', 'UTC_Time_Measured_2_Pacific':'Time (Pacific)', 'staff_gauge_ht_ft':'Outside Staff Reading (ft)', 'data_downloaded':'Downloaded Data', 'logger_serial':'Logger Serial#', 'time_logger_pulled_UTC_time_Pacific':'Time Logger Pulled (Pacific)', 'time_logger_re_deployed_UTC_t_Pacific':'Time Logger Re-deployed (Pacific)', 'battery':'Battery', 'additional_notes':'Notes', 'globalid':'GlobalID', 'SHAPE':'SHAPE'}, inplace= True)

In [115]:
sedfHydroStaffGageData


,Date,Time Gauge Reading (Pacific),Observers,Other Observers,Water Level (ft),Time (Pacific),Outside Staff Reading (ft),Downloaded Data,Logger Serial#,Time Logger Pulled (Pacific),Time Logger Re-deployed (Pacific),Battery,Notes,GlobalID,SHAPE
0,2/18/2020,12:30:00 PM,ryan_m,None,1.69,NaN,NaN,no,None,NaN,NaN,None,Water cold,bc3befc7-bbf8-421b-b631-75f4eb7eafda,"{""x"": -123.02104645394677, ""y"": 46.96475303988..."
1,2/18/2020,NaN,ryan_m,None,NaN,12:34:09 PM,1.69,yes,10766585,12:33:00 PM,12:38:00 PM,None,Looks good! Water in logger but all loggers d...,6aa96015-f495-4f61-bcdf-d0abe8a54858,"{""x"": -123.02099369477762, ""y"": 46.96459505711..."
9,2/21/2020,3:00:00 PM,ryan_m,None,1.40,NaN,NaN,no,None,NaN,NaN,None,"Drive by survey, Binos from road, gauge needs ...",f0e75a3e-d627-4954-8daa-7bf1f9becb35,"{""x"": -123.02103013266245, ""y"": 46.96476852729..."
11,2/27/2020,2:59:56 PM,ryan_m,None,1.29,NaN,NaN,no,None,NaN,NaN,None,"55 plus, sunny no wind, frog survey, just gett...",3d5e86d4-0cb4-4425-8899-e107ce68be86,"{""x"": -123.02105458439287, ""y"": 46.96477024963..."
21,3/6/2020,1:21:43 PM,ryan_m,None,1.35,NaN,NaN,no,None,NaN,NaN,None,None,5ef59a89-ab85-4c08-acb4-60244f27a46c,"{""x"": -123.0209257175429, ""y"": 46.964550088203..."
31,3/21/2020,3:43:43 PM,ryan_m,None,0.78,NaN,NaN,no,None,NaN,NaN,None,"Clean gauge, removed veg",731cc50f-5dd9-4692-9db8-2b3897cd50c2,"{""x"": -123.02094281662536, ""y"": 46.96458072405..."
33,5/22/2020,10:34:50 AM,ryan_m,None,0.56,NaN,NaN,no,None,NaN,NaN,None,"Stream flowing, forgot key and tools for data ...",c3dca931-64da-40aa-9f5f-18d2c1abedbd,"{""x"": -123.02096201118364, ""y"": 46.96458667521..."
44,5/27/2020,6:17:00 PM,ryan_m,None,0.45,NaN,NaN,None,None,NaN,NaN,None,"Installed 3 exclosures on piez 1-3, amazing we...",33404686-f3c9-452b-8d2d-b22b9695f4ce,"{""x"": -123.02094281662536, ""y"": 46.96461027026..."
45,5/27/2020,NaN,ryan_m,None,NaN,6:21:00 PM,0.45,yes,10766585,6:21:00 PM,6:24:00 PM,None,Everything is good,d05ba08a-b909-4f2d-938c-4f2c74454fb2,"{""x"": -123.0209698901726, ""y"": 46.964596314399..."
50,6/3/2020,6:49:05 PM,ryan_m,None,0.30,NaN,NaN,no,None,NaN,NaN,None,"Quick survey! Used Bono’s, very accurate",86a2bd5a-f4bc-408b-9dc9-55ab3c78b63e,"{""x"": -123.02097106363904, ""y"": 46.96454795081..."


In [130]:
## Create export paths
writer = pd.ExcelWriter(os.path.join(out_workspace,(timestamp + '_WilsonDairyHydro.xlsx')))
sedfHydroAllData.to_excel(writer, 'StaffGage')
writer.save()

ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

In [167]:
sedfHydroAllData

,objectid,globalid,CreationDate,Creator,EditDate,Editor,observers,observers_other,todays_date,Measurement_Type,...,SHAPE,CreationDate_Pacific,EditDate_Pacific,todays_date_Pacific,UTC_Time_Measured_4_Pacific,UTC_Time_Measured_1_Pacific,UTC_Time_Measured_2_Pacific,time_logger_pulled_UTC_time_Pacific,time_logger_re_deployed_UTC_t_Pacific,UTC_Time_Measured_3_Pacific
0,9,bc3befc7-bbf8-421b-b631-75f4eb7eafda,2/19/2020 1:27:35 AM,ryan_munes@fws.gov_fws,3/4/2020 8:01:31 PM,ryan_munes@fws.gov_fws,ryan_m,None,2/18/2020 8:00:00 PM,dempsey_creek_level_quick_surve,...,"{""x"": -123.02104645394677, ""y"": 46.96475303988...",2/18/2020 5:27:35 PM,3/4/2020 12:01:31 PM,2/18/2020 12:00:00 PM,2/18/2020 12:30:00 PM,NaN,NaN,NaN,NaN,NaN
1,10,6aa96015-f495-4f61-bcdf-d0abe8a54858,2/19/2020 1:27:41 AM,ryan_munes@fws.gov_fws,2/19/2020 1:27:41 AM,ryan_munes@fws.gov_fws,ryan_m,None,2/18/2020 8:00:00 PM,piezometers,...,"{""x"": -123.02099369477762, ""y"": 46.96459505711...",2/18/2020 5:27:41 PM,2/18/2020 5:27:41 PM,2/18/2020 12:00:00 PM,NaN,NaN,2/18/2020 12:34:09 PM,2/18/2020 12:33:00 PM,2/18/2020 12:38:00 PM,NaN
2,11,7d52d096-c005-4185-b836-9ba34954778e,2/19/2020 1:27:44 AM,ryan_munes@fws.gov_fws,2/19/2020 1:27:44 AM,ryan_munes@fws.gov_fws,ryan_m,None,2/18/2020 8:00:00 PM,piezometers,...,"{""x"": -123.02116820600163, ""y"": 46.96437054783...",2/18/2020 5:27:44 PM,2/18/2020 5:27:44 PM,2/18/2020 12:00:00 PM,NaN,NaN,NaN,2/18/2020 12:26:40 PM,2/18/2020 12:27:00 PM,NaN
3,12,eabd64f8-731d-4936-89b5-48c308cc3d6b,2/19/2020 1:27:48 AM,ryan_munes@fws.gov_fws,2/19/2020 1:27:48 AM,ryan_munes@fws.gov_fws,ryan_m,None,2/18/2020 8:00:00 PM,piezometers,...,"{""x"": -123.02116820600163, ""y"": 46.96437054783...",2/18/2020 5:27:48 PM,2/18/2020 5:27:48 PM,2/18/2020 12:00:00 PM,NaN,NaN,NaN,2/18/2020 12:23:44 PM,2/18/2020 12:24:00 PM,NaN
4,13,9cbc9992-92ff-4a84-bde2-2fb62f75998a,2/19/2020 1:27:55 AM,ryan_munes@fws.gov_fws,2/19/2020 5:24:42 AM,ryan_munes@fws.gov_fws,ryan_m,None,2/18/2020 8:00:00 PM,piezometers,...,"{""x"": -123.02195627253784, ""y"": 46.96454610679...",2/18/2020 5:27:55 PM,2/18/2020 9:24:42 PM,2/18/2020 12:00:00 PM,NaN,2/18/2020 12:00:34 PM,NaN,2/18/2020 12:03:00 PM,2/18/2020 12:11:21 PM,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,215,9840d702-50ec-4fcf-ad92-0f547f60c996,4/17/2021 2:19:16 PM,ryan_munes@fws.gov_fws,4/17/2021 2:19:16 PM,ryan_munes@fws.gov_fws,ryan_m,None,4/15/2021 7:00:00 PM,dempsey_creek_level_quick_surve,...,"{""x"": -123.02096318465009, ""y"": 46.96457452145...",4/17/2021 7:19:16 AM,4/17/2021 7:19:16 AM,4/15/2021 12:00:00 PM,4/15/2021 10:48:00 AM,NaN,NaN,NaN,NaN,NaN
203,216,a8f9c5c1-1c77-41e5-87fe-910aa53d065b,4/26/2021 4:31:53 AM,ryan_munes@fws.gov_fws,4/26/2021 4:31:53 AM,ryan_munes@fws.gov_fws,ryan_m,None,4/26/2021 2:14:19 AM,dempsey_creek_level_quick_surve,...,"{""x"": -123.02071290102135, ""y"": 46.96460515730...",4/25/2021 9:31:53 PM,4/25/2021 9:31:53 PM,4/25/2021 7:14:19 PM,4/25/2021 7:14:00 PM,NaN,NaN,NaN,NaN,NaN
204,217,ca7d30f4-e59f-4cff-b083-34037c06c897,4/26/2021 11:38:29 PM,ryan_munes@fws.gov_fws,4/26/2021 11:38:29 PM,ryan_munes@fws.gov_fws,ryan_m,None,4/26/2021 2:21:06 AM,culverts,...,"{""x"": -123.02210479986203, ""y"": 46.96474580564...",4/26/2021 4:38:29 PM,4/26/2021 4:38:29 PM,4/25/2021 7:21:06 PM,NaN,NaN,NaN,NaN,NaN,4/25/2021 7:21:17 PM
205,218,a51fbed3-73c3-48c4-85b8-ff1ec1d086da,4/26/2021 11:38:36 PM,ryan_munes@fws.gov_fws,4/26/2021 11:38:36 PM,ryan_munes@fws.gov_fws,ryan_m,None,4/26/2021 2:23:46 AM,culverts,...,"{""x"": -123.02222373906804, ""y"": 46.96474991277...",4/26/2021 4:38:36 PM,4/26/2021 4:38:36 PM,4/25/2021 7:23:46 PM,NaN,NaN,NaN,NaN,NaN,4/25/2021 7:23:00 PM
